In [2]:
import base64
import datetime
import requests
import json
import pandas as pd
import time

In [3]:
import secrets

In [4]:
credentials = "%s:%s" % (secrets.paypal_client_id, secrets.paypal_secret)
auth_creds = base64.b64encode(credentials.encode('utf-8')).decode('utf-8').replace("\n", "")

In [51]:
api_endpoint = "https://api.paypal.com/v1"
#api_endpoint = "https://api.sandbox.paypal.com/v1"

In [60]:
def get_auth():
    headers = {
        "Accept": "application/json",
        "Accept-Language": "en_US",
        "Authorization": "Basic " + auth_creds
    }

    data = {
        "grant_type": "client_credentials"
    }

    resp = requests.post(api_endpoint + "/oauth2/token",
                         headers=headers,
                         data=data)
    return resp

In [67]:
auth = get_auth()
print(auth.status_code)
auth = auth.json()

401


In [68]:
auth

{'error': 'invalid_client',
 'error_description': 'Client Authentication failed'}

In [63]:
def get(endpoint, data=None):
    if endpoint.startswith("http"):
            url = endpoint
    elif endpoint.startswith("/"):
        url = "{}{}".format(api_endpoint, endpoint)
    else:
        url = "{}/{}".format(api_endpoint, endpoint)

    response = requests.get(
            url,
            data=data,
            headers= {
                "Content-Type": "application/json",
                "Accept": "application/json",
                "Authorization": "Bearer " + auth['access_token'],
            }
        )
    if response.status_code == 200:
            return response.json()
        
    elif response.status_code == 401:
        print("401: Unauthorized")
    elif response.status_code == 404:
        print("404: Probably invalid endpoint")
    else:
        print("ERROR IN REQUEST: {}".format(response.content))
    return response

In [64]:
get('payments/payment')

TypeError: byte indices must be integers or slices, not str